In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
from pathlib import Path
from typing import Any

import torchvision.transforms.v2.functional as v2F  # type: ignore
from torchvision import tv_tensors
from tqdm.auto import tqdm, trange

import bb
import bbedit
import tt
import tt_fcos

LOG = logging.getLogger(__name__)
tt.logging_init()

In [4]:
CATEGORIES = [
    "chicken",
    "cow",
    "creeper",
    "enderman",
    "pig",
    "sheep",
    "skeleton",
    "spider",
    "zombie",
]
CATEGORIES.sort()

DATA_PATH = Path.home() / "data"

# Create dataset with set of images

In [ ]:
# Create a new dataset and copy in images
new = bb.MCDataset.new(
    dset_dir=DATA_PATH / "new",
    categories=CATEGORIES,
    input_images_dir=DATA_PATH / "minecraft-mobs/images",
)

# Run new dataset through latest model

In [ ]:
# Load model
model_path = DATA_PATH / "checkpoints/keep/best.pt"
trainer = tt_fcos.FCOSTrainer.load_checkpoint(model_path, project_dir=model_path.parent, score_thresh=0.5)

# Load dataset
new_path = DATA_PATH / "new"
new_dset = bb.MCDataset(new_path)

In [ ]:
# Run infer on images and display the results
bb.BBoxViewer(new_dset, infer_fn=trainer.infer).show_widget()

In [ ]:
# Run each image through the model and save the results in the dataset
for idx in trange(len(new_dset)):                                                                               
    image, target = new_dset[idx]                                                                                   
    det = trainer.infer(image)
    new_dset.add_annotation(idx, det)                                                                     
new_dset.rebuild_index()

In [ ]:
# Show targets saved in the dataset
new_dset.view()

In [ ]:
# Save model predictions
new_dset.save_annotations()

# Hand classify

In [5]:
DATA_PATH = Path.home() / "data"
new_dset_path = DATA_PATH / "new"

In [6]:
new_dset = bb.MCDataset(new_dset_path)
print(new_dset.image_info(5))
new_dset[5][1]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
{'id': 5, 'file_name': 'frame_199695.png', 'width': 640, 'height': 640}


{'image_id': 5,
 'boxes': BoundingBoxes([[283.8500, 408.9593, 449.9167, 628.7106]], format=BoundingBoxFormat.XYXY, canvas_size=(640, 640), clamping_mode=soft),
 'labels': tensor([2])}

In [19]:
x = new_dset[0][1]
x

{'image_id': 0,
 'boxes': BoundingBoxes([[230.5681, 231.9958, 306.1765, 389.8882],
                [455.4760, 135.3701, 498.1576, 177.4611],
                [616.8823, 157.6265, 639.3231, 224.5572]], format=BoundingBoxFormat.XYXY, canvas_size=(640, 640), clamping_mode=soft),
 'labels': tensor([2, 1, 1])}

In [6]:
bbedit.DEBUG.clear_output()
bbedit.DEBUG

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [21]:
bbe = bbedit.BBoxEdit(new_dset_path)
bbe.display()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


KeyError: 'h'

In [ ]:
check_dset = bb.Dataset.load(new_dset_path)
check_dset.view()

# Stats

In [ ]:
data_dir = Path.home() / "src/data"
dset = bb.Dataset.load(data_dir / "minecraft/info.json")
display(dset.dataset_stats())
display(dset.category_stats())